Environment: This script should be run with the `r_environment_pathogen_atlas` environment using the devcontainer `devcontainer_R` 

In [ ]:
library(spacexr)
library(Matrix)
library(doParallel)
library(ggplot2)
library(RcppZiggurat)

In [ ]:
data_folder <- '../../data'
outputs_folder <- '../../outputs'

In [ ]:
datadir <- file.path(data_folder, 'cside_preparations', '9hr_avr')
### mk directory for outputs
if (!dir.exists(file.path(outputs_folder, 'CSIDE_outputs'))) {
  dir.create(file.path(outputs_folder, 'CSIDE_outputs'))
}
savedir <- file.path(outputs_folder, 'CSIDE_outputs', 'RCTD_results_9hr')

dir.create(savedir)

Preparing RCTD deconvolution

In [ ]:

counts <- read.csv(file.path(datadir,"counts.csv")) # load in counts matrix
coords <- read.csv(file.path(datadir,"coords.csv"))
rownames(counts) <- counts[,1]; counts[,1] <- NULL # Move first column to rownames
rownames(coords) <- coords$barcodes; coords$barcodes <- NULL # Move barcodes to rownames
nUMI <- colSums(counts) # In this case, total counts per pixel is nUMI
puck <- SpatialRNA(coords, counts, nUMI)
barcodes <- colnames(puck@counts) # pixels to be used (a list of barcode names). 
plot_puck_continuous(puck, barcodes, puck@nUMI, ylimit = c(0,round(quantile(puck@nUMI,0.9))), 
                   title ='plot of nUMI') 

refdir <- file.path(data_folder, 'cside_preparations', 'scrna')
counts <- read.csv(file.path(refdir,"dge.csv")) # load in counts matrix
rownames(counts) <- counts[,1]; counts[,1] <- NULL # Move first column to rownames
meta_data <- read.csv(file.path(refdir,"meta_data.csv")) # load in meta_data (barcodes, clusters, and nUMI)
cell_types <- meta_data$cluster; names(cell_types) <- meta_data$barcode # create cell_types named list
cell_types <- as.factor(cell_types) # convert to factor data type
nUMI <- meta_data$nUMI; names(nUMI) <- meta_data$barcode # create nUMI named list
reference <- Reference(counts, cell_types, nUMI, min_UMI = 15)
myRCTD <- create.RCTD(puck, reference, max_cores = 16, gene_cutoff = 0.0001, gene_cutoff_reg = 0.0001, fc_cutoff = -3, fc_cutoff_reg = -3)
myRCTD <- run.RCTD(myRCTD, doublet_mode = 'doublet')
saveRDS(myRCTD,file.path(savedir,'myRCTD.rds'))


In [ ]:
t_var <- read.csv(file.path(datadir,"axis.csv"))
t_var <-setNames(t_var$Smoothed, t_var$X)

Plotting the explanatory variable

In [ ]:
### Create SpatialRNA object
myRCTD <- readRDS(file.path(savedir,'myRCTD.rds'))
set.seed(12345)
explanatory.variable <- t_var
print(head(explanatory.variable))

plot_puck_continuous(myRCTD@spatialRNA, names(explanatory.variable), explanatory.variable, ylimit = c(0, 1), title ='plot of explanatory variable') 

Running C-SIDE differential expression

In [ ]:
#de
myRCTD@config$max_cores <- 16
myRCTD <- run.CSIDE.single(myRCTD, explanatory.variable, 
                        cell_type_threshold = 0) 

saveRDS(myRCTD,file.path(savedir,'myRCTDde.rds'))

Plotting mesophyll results

In [ ]:
#print results for cell type Mesophyll ('2')
cell_type <- '2'
results_de <- myRCTD@de_results$sig_gene_list[[cell_type]]
print(results_de)

Saving object and plots

In [ ]:
myRCTD <- readRDS(file.path(savedir,'myRCTDde.rds'))
#plot_gene_two_regions(myRCTD, sig_gene, cell_type)

In [ ]:
make_all_de_plots(myRCTD, savedir)